In [1]:
import numpy as np
import math

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
%matplotlib inline
# %matplotlib

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
def sigmoid(x, derivative=False):
    prob = 1 / ( 1 + np.exp(-x))
    
    if derivative:
        d_sigmoid = prob * (1 - prob)
        return d_sigmoid
    return prob

def h_theta(x, w, b):
    z = (x @ w) + b
    return sigmoid(z)

In [3]:
from helper.utils import plot_h_theta_1d
interactive_plot = interact(plot_h_theta_1d,
         b=widgets.FloatSlider(min=-5, max=5, step=0.5, value=0),
         w=widgets.FloatSlider(min=-5, max=5, step=1, value=1) );

interactive(children=(FloatSlider(value=0.0, description='b', max=5.0, min=-5.0, step=0.5), FloatSlider(value=…

In [8]:
from helper.utils import plot_linear_hyperplane
interact(plot_linear_hyperplane,
         b=widgets.FloatSlider(min=-5, max=5, step=1, value=0),
         w_magnitude=widgets.FloatSlider(min=0, max=3, step=0.2, value=2),
         w_angle=widgets.FloatSlider(min=-np.pi, max=np.pi, step=np.pi/8, value=0));

interactive(children=(FloatSlider(value=0.0, description='b', max=5.0, min=-5.0, step=1.0), FloatSlider(value=…

In [5]:
from helper.utils import plot_linear_logistic_2d
interact(plot_linear_logistic_2d,
         b=widgets.FloatSlider(min=-5, max=5, step=1, value=0),
         w_magnitude=widgets.FloatSlider(min=0.5, max=5, step=0.5, value=2),
         w_angle=widgets.FloatSlider(min=-np.pi, max=np.pi, step=0.5, value=0) );

interactive(children=(FloatSlider(value=0.0, description='b', max=5.0, min=-5.0, step=1.0), FloatSlider(value=…

In [6]:
from sklearn.datasets import load_breast_cancer, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
data = load_breast_cancer()
X = data['data']
y = data['target']
print(X.shape)
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling (standardize the data)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


(569, 30)


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.animation import FuncAnimation

class LogisticRegression:
    def __init__(self, learning_rate=0.01, max_iter=1000, batch_size=32):
        # Initialize parameters
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.weights = None  # To be initialized in fit()
        self.losses = []  # To keep track of loss over iterations

    def _sigmoid(self, z):
        # Sigmoid activation function
        return 1 / (1 + np.exp(-z))
    
    def _h_theta(self, X):
        # Hypothesis function: sigmoid(X * weights)
        return self._sigmoid(np.dot(X, self.weights))
    
    def _cross_entropy_loss(self, h, y):
        # Cross-entropy loss function
        return - np.mean(y * np.log(h) + (1 - y) * np.log(1 - h))
    
    def _compute_loss_and_gradient(self, X, y):
        # Compute loss and gradient for current weights
        m = X.shape[0]
        h = self._h_theta(X)
        loss = self._cross_entropy_loss(h, y)
        grad = np.dot(X.T, (h - y)) / m  # Gradient of loss w.r.t. weights
        return loss, grad

    def plot_decision_boundary(self, X, y, iteration):
        # Create a meshgrid to cover the feature space
        x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
        y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                             np.arange(y_min, y_max, 0.01))

        # Predict probabilities for each point in the meshgrid
        Z = self.predict(np.c_[xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        # Plot contour and training examples
        plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.coolwarm)
        plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', marker='o', s=50, cmap=plt.cm.coolwarm)
        plt.title(f'Logistic Regression Decision Boundary - Iteration {iteration}')
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')

        # Save the plot
        plt.savefig(f'frames/decision_boundary_{iteration}.png')
        plt.close()

    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.zeros(n)  # Initialize weights to zeros

        # Create directory to store frames if it doesn't exist
        if not os.path.exists('frames'):
            os.makedirs('frames')

        for i in range(self.max_iter):
            # Shuffle the data correctly
            indices = np.random.permutation(m)
            X_shuffled = X[indices]  # Shuffle rows of X
            y_shuffled = y[indices]  # Shuffle rows of y (same order)

            # Mini-batch gradient descent
            for start in range(0, m, self.batch_size):
                end = start + self.batch_size
                batch_X = X_shuffled[start:end]  # Slice the shuffled batch of X
                batch_y = y_shuffled[start:end]  # Slice the shuffled batch of y
                
                if len(batch_X) == 0:  # Safety check for empty batch
                    continue

                loss, grad = self._compute_loss_and_gradient(batch_X, batch_y)
                self.weights -= self.learning_rate * grad  # Update weights
            
            # Calculate loss on the full dataset after each epoch
            epoch_loss, _ = self._compute_loss_and_gradient(X, y)
            self.losses.append(epoch_loss)

            # Capture and save the decision boundary every 100 iterations
            if i % 10 == 0:
                # print(f"Loss at iteration {i}: {epoch_loss:.4f}")
                self.plot_decision_boundary(X, y, i)

        return self
    
    def predict_proba(self, X):
        # Predict probabilities
        return self._h_theta(X)
    
    def predict(self, X):
        # Predict binary labels
        return np.round(self.predict_proba(X))
    
    def plot_loss(self):
        # Plot the loss over iterations
        plt.plot(range(len(self.losses)), self.losses, marker='o')
        plt.title("Loss over iterations")
        plt.xlabel("Iteration")
        plt.ylabel("Loss")
        plt.grid(True)
        plt.show()


# Example usage with training
# Assuming you have your training data ready: X_train, y_train
model = LogisticRegression(learning_rate=0.02, max_iter=1000, batch_size=8)
model.fit(X_train, y_train)


ValueError: shapes (696426,2) and (30,) not aligned: 2 (dim 1) != 30 (dim 0)

In [ ]:
import imageio

# Create a GIF from the saved frames
images = []
for i in range(0, 1000, 10):  # Adjust step based on how frequently you saved frames
    filename = f'frames/decision_boundary_{i}.png'
    images.append(imageio.imread(filename))

imageio.mimsave('decision_boundary_evolution1.gif', images, fps=5)  # Save GIF


In [ ]:
# !pip install imageio